In [ ]:
%pip install pypdfium2 -q
%pip install requests -q
%pip install python-dotenv -q

In [ ]:
import os
import requests
import base64
import json

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Configuration
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
GPT4V_DEPLOYMENT = os.getenv("GPT4_V_DEPLOYMENT")
API_VERSION = os.getenv("OPENAI_VISION_API_VERSION")

GPT4V_ENDPOINT = f"{AOAI_ENDPOINT}openai/deployments/{GPT4V_DEPLOYMENT}/extensions/chat/completions?api-version={API_VERSION}"
GPT4V_KEY = os.getenv("AZURE_OPENAI_API_KEY")

headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

## probar un nuevo system prompt - Cartas Reyes Magos


In [ ]:
filenames = []
filenames.append("Card_094840789004155103033.jpg")
filenames.append("Card_094840789004155103036.jpg")
filenames.append("Card_094840789004580103028.jpg")
filenames.append("Carta 2.jpg")
filenames.append("Carta 3.jpg")

system_msg = "Eres un analista de textos manuscritos por niños, relacionados con la carta que los niños escriben a los reyes magos para pedir juguetes."

user_msg = """Analiza el contenido de la imagen y devuélveme el texto que contiene, de una forma ordenada y estructurada como si fuera un informe. Además, quiero que hagas los siguientes análisis: 
1 - Analiza qué regalos son juguetes bélicos, complementos de móvil o patinetes eléctricos y si cuestan más de 50€.
2 - Analiza si las cartas tienen un dibujo y cuantas filas han escrito. 
3 - Analiza si tienen datos confidenciales, considerando como datos confidenciales apellidos de persona o direcciones postales. Únicamente los datos de nombre y edad no son confidenciales.
Para cada uno de los puntos analizados, devuelve un resultado del análisis y una conclusión.
"""

for filename in filenames:

    data = {
        "model": "gpt-4-vision-preview",
        "enhancements": {
            "ocr": {
            "enabled": True
            },
            "grounding": {
            "enabled": False
            }
        },
        "dataSources": [
        {
            "type": "AzureComputerVision",
            "parameters": {
                "endpoint": os.getenv("COMPUTER_VISION_ENDPOINT"),
                "key": os.getenv("COMPUTER_VISION_API_KEY")
            }
        }],
        "messages": [ 
            { "role": "system", "content": f"{system_msg}" }, 
            { "role": "user", 
            "content": [
                {
                    "type": "text",
                    "text": f"{user_msg}"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "detail": "high",
                        "url": f"https://url-to-file/{filename}"
                    }
                }
            ]} 
        ], 
        "max_tokens": 2000 
    }

    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=data)

    # print(response.json())
    # print(response.json()["usage"]["total_tokens"])

    result = response.json()["choices"][0]["message"]["content"]

    # Specify the file path
    output_file = f"data/{filename}.txt"

    # Open the file in write mode
    with open(output_file, "w", encoding="UTF-8") as file:
        # Write the result to the file
        file.write(result)

    print("Result stored in", output_file)

## probar un nuevo system prompt - Cartas CEO

Podríamos repetir la prueba por ejemplo de las de adulto (las de niños no hace falta) poniendo de alguna manera el contexto rol? 

"Soy una analista financiero, trabajo en un banco, en mi contexto sobre todo utilizo un lenguaje profesional y formal, haciendo referencias al entorno macroeconómico, microeconómico, transformación digital, ciberseguridad, normas, normativas, regulación, sostenibilidad y personas, liderazgo y recursos humanos. Cuando interpretes mis notas ten en cuenta este contexto para elegir la palabra más adecuada"



In [ ]:
filenames = []
filenames.append("Imagen-Notas-Clara-Letra 1.pdf_000.jpg")
filenames.append("Imagen-Notas-Clara-Letra 2.pdf_000.jpg")
filenames.append("Notas - Manuscritas.pdf_000.jpg")

system_msg = "Eres un analista financiero, trabajas para un banco, y usas un lenguaje profesional y formal, haciendo referencia a entornos macroeconómicos, microeconómicos, transformación digital, ciberseguridad, normas, normativas, regulación, sostenibilidad y personas, liderazgo y recursos humanos. Cuando interpretes notas manuscritas ten en cuenta el contexto para elegir la palabra más adecuada."

user_msg = "Analiza el contenido de la imagen y devuélveme el texto que contiene, de una forma ordenada y estructurada como si fuera un informe."

for filename in filenames:

    data = {
        "model": "gpt-4-vision-preview",
        "enhancements": {
            "ocr": {
            "enabled": True
            },
            "grounding": {
            "enabled": False
            }
        },
        "dataSources": [
        {
            "type": "AzureComputerVision",
            "parameters": {
                "endpoint": os.getenv("COMPUTER_VISION_ENDPOINT"),
                "key": os.getenv("COMPUTER_VISION_API_KEY")
            }
        }],
        "messages": [ 
            { "role": "system", "content": f"{system_msg}" }, 
            { "role": "user", 
            "content": [
                {
                    "type": "text",
                    "text": f"{user_msg}"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "detail": "high",
                        "url": f"https://url-to-file/{filename}"
                    }
                }
            ]} 
        ], 
        "max_tokens": 2000 
    }

    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=data)

    # print(response.json())
    # print(response.json()["usage"]["total_tokens"])
    
    result = response.json()["choices"][0]["message"]["content"]

    # Specify the file path
    output_file = f"data/{filename}.txt"

    # Open the file in write mode
    with open(output_file, "w", encoding="UTF-8") as file:
        # Write the result to the file
        file.write(result)

    print("Result stored in", output_file)